# Google Colab (Skip locally)

In [ ]:
#@title Mount your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title Link your assignment folder & install requirements
#@markdown Enter the path to the assignment folder in your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
import sys
import os
import shutil
import warnings

folder = "/content/gdrive/MyDrive/IFT6135/HW2_2025/src" #@param {type:"string"}
!ln -Ts "$folder" /content/src 2> /dev/null

# Add the assignment folder to Python path
if '/content/src' not in sys.path:
  sys.path.insert(0, '/content/src')

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
  warnings.warn('CUDA is not available.')

# Imports & Setup

In [3]:
%matplotlib inline
import warnings

from main import train_models
from arguments import Arguments
import os
from checkpointing import get_all_checkpoints_per_trials
from plotter import plot_metrics_across_configurations, plot_best_across_configurations

models = ['lstm', 'gpt']
seeds = [0, 42, 7]
colab = False

ImportError: cannot import name 'get_plot_data' from 'metrics' (c:\Users\fwilh\Desktop\IFT6135-HW2\src\metrics.py)

# Experiments

## Experiment 1 (Sanity Check)

In [ ]:
experiment1 = Arguments()
if colab: experiment1.num_workers = 2
experiment1.log_dir = "logs/experiment1"

for model in models:
    experiment1.model = model
    experiment1.exp_name = model
    train_models(experiment1)

In [ ]:
!zip -r experiment1.zip logs/experiment1

## Experiment 2 (Scaling Data Size : Training Ratio)

In [ ]:
experiment2 = Arguments()
log_dir = "logs/experiment2"
experiment2.verbose = False

for model in models:
    experiment2.model = model
    experiment2.log_dir = os.path.join(log_dir, model)
    for r in range(1, 10):
        r = r / 10
        experiment2.r_train = r
        experiment2.exp_name = f"{model}_r_{r}"
        train_models(experiment2)

In [ ]:
!zip -r experiment2.zip logs/experiment2

In [2]:
experiment2 = Arguments()
experiment2.verbose = False

experiment = experiment2
log_dir = "../logs/experiment2"
configurations = [r/10 for r in range(1, 10)]
labels = [f"r_{r}" for r in configurations]

results = {}
for model in models:
    results[model] = {}
    for label, configuration in zip(labels, configurations):
        exp_name = f"{model}_{label}"
        experiment.log_dir = os.path.join(log_dir, model)
        checkpoints = []
        for m, seed in enumerate([0, 42]):
            checkpoint_path = os.path.join(log_dir, model, exp_name, str(m))
            checkpoints.append(checkpoint_path)
        
        models_path, metrics = get_all_checkpoints_per_trials(checkpoints, exp_name, just_files=True, verbose=experiment.verbose)
        results[model][label] = metrics
    
    plot_metrics_across_configurations(results[model], experiment.log_dir, "std")
    plot_best_across_configurations(results[model], experiment.log_dir)

NameError: name 'models' is not defined

In [ ]:
import matplotlib.pyplot as plt

def plot_accuracy_and_step_on_same_plot(config_names, best_accuracies, best_accuracy_steps):
    """
    Plot best accuracy and the step of best accuracy on the same figure using two y-axes.
    
    Parameters:
        config_names (list): List of configuration names (strings) for the x-axis.
        best_accuracies (list): Best accuracy values, one per configuration.
        best_accuracy_steps (list): Step at which the best accuracy was reached, one per configuration.
    """
    # Create a figure and an axis
    fig, ax1 = plt.subplots(figsize=(8,6))

    # Plot best accuracy on the first y-axis
    color1 = 'tab:blue'
    ax1.set_xlabel('Configuration')
    ax1.set_ylabel('Best Accuracy', color=color1)
    ax1.plot(config_names, best_accuracies, marker='o', color=color1, label='Best Accuracy')
    ax1.tick_params(axis='y', labelcolor=color1)
    
    # Create a twin of the first axis to share the same x-axis
    ax2 = ax1.twinx()
    color2 = 'tab:red'
    ax2.set_ylabel('Step of Best Accuracy', color=color2)
    ax2.plot(config_names, best_accuracy_steps, marker='x', color=color2, label='Step of Best Accuracy')
    ax2.tick_params(axis='y', labelcolor=color2)

    # Optionally, add a combined legend. We'll manually create a legend from both lines.
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()
    ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='best')
    
    plt.title('Best Accuracy and Step vs. Configuration')
    plt.tight_layout()
    plt.show()



plot_accuracy_and_step_on_same_plot(configs, accuracies, accuracy_steps)


## Experiment 3 (Scaling Data Size : P)

In [ ]:
experiment3 = Arguments()
experiment3.log_dir = "logs/experiment3"
experiment3.verbose = False

experiment3.p = 11
experiment3.operation_orders = [2, 3]

for model in models:
    experiment3.model = model
    
    experiment3.exp_name = model
    train_models(experiment3)

In [ ]:
!zip -r experiment3.zip logs/experiment3

In [ ]:
log_dir = "../logs/experiment3"
verbose = False

performances = {}
results = {}
for model in models:
    results[model], performances[model] = {}, {}
    exp_name = f"{model}"
    checkpoints = []
    for m, seed in enumerate([0, 42]):
        checkpoint_path = os.path.join(log_dir, model, str(m))
        checkpoints.append(checkpoint_path)
    
    models_path, metrics = get_all_checkpoints_per_trials(checkpoints, exp_name, just_files=True, verbose=experiment.verbose)
    results[model] = metrics
    
plot_all(results, save_path=log_dir, mode="std")

## Experiment 4 (Scaling Model Size)

In [ ]:
experiment4 = Arguments()
logdir = "logs/experiment4"
experiment4.verbose = False

for model in models:
    experiment4.model = model
    experiment4.log_dir = os.path.join(log_dir, model)
    for L in [1, 2, 3]:
        experiment4.num_layers = L
        for d in [2**6, 2**7, 2**8]:
            experiment4.embedding_size = d
            experiment4.hidden_size = d
            experiment4.exp_name = f"{model}_L_{L}_d_{d}"
            train_models(experiment4)

In [ ]:
!zip -r experiment4.zip logs/experiment4

## Experiment 5 (Scaling Compute)

In [ ]:
experiment5 = Arguments()
log_dir = "logs/experiment5"
experiment5.verbose = False

experiment5.n_steps = 2 * 10**4 + 1

for model in models:
    experiment5.model = model
    experiment5.log_dir = os.path.join(log_dir, model)
    for B in [2**5, 2**6, 2**7, 2**8, 2**9]:
        experiment5.batch_size = B

        experiment5.exp_name = f"{model}_B_{B}"
        train_models(experiment5)

In [ ]:
!zip -r experiment5.zip logs/experiment5

## Experiment 6 (Regularisation)

In [ ]:
experiment6 = Arguments()
log_dir = "logs/experiment6"
experiment6.verbose = False

experiment6.n_steps = 4 * 10**4 + 1
model = "lstm"
experiment6.model = model
experiment6.log_dir = os.path.join(log_dir, model)

for wd in [0.25, 0.5, 0.75, 1.0]:
    experiment6.weight_decay = wd

    experiment6.exp_name = f"{model}_wd_{wd}"
    train_models(experiment6)

In [ ]:
!zip -r experiment6.zip logs/experiment6